In [ ]:
#X_train : label encoded
#X_train_1 : one hot encoded
#X_train_scale : Scaled data
#X_train_orig: original data without enocding 
#outlier_predict_train : SVM train result
#outlier_predict_test: SVM test result

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import svm
from numpy import array

In [ ]:
#Get the dataset
df=pd.read_csv('nginx_IP_validation')

In [ ]:
#Select only required features
ndf=df[['ip','response_code','url','access_method','user_name','user_agent_device','user_agent_os','user_agent_name','geoip_city_name','geoip_continent_name','geoip_country_iso_code','geoip_region_name','action']]

#preprocessing begins
#splitting ip into 4 segments
#ip=ndf['ip']
#ip_df=pd.DataFrame(ip)
#ip_df=ip_df['ip'].str.split('.',expand=True).rename(columns=lambda x: "segment"+str(x+1)).astype(int)
#ndf=ndf.join(ip_df)

#getting the first accessed route from url
url=ndf['url']
url_df=pd.DataFrame(url)
url_df=url_df['url'].str.split('/',expand=True)
url_df=url_df.loc[:,1].to_frame('url')
url_df=url_df.replace(r'^\s*$', np.nan, regex=True)
url_df.url.fillna('events',inplace=True)
url_df.rename(columns={'url':'access_route'},inplace=True)
ndf=ndf.join(url_df)

In [ ]:
#Converting IP address to int values
ip_transform=[]
from ipaddress import ip_address
for i in df['ip']:
    ips=int(ip_address(i))
    ip_transform.append(ips)

ip_df=pd.DataFrame(ip_transform)
ip_df.rename(columns={0:'ip_transform'},inplace=True)
ndf=ndf.join(ip_df)

In [ ]:
#Making allow as 1 and block as -1 and storing it into attack column
ndf.loc[ndf['action'] == "allow", "attack"] = 1  
ndf.loc[ndf['action'] != "allow", "attack"] = -1
target=ndf['attack']

In [ ]:
#drop the processed columns
ndf.drop(columns=['ip','url','action'],inplace=True)

In [ ]:
#estimate number of outliers in the dataset
nu = target[target == -1]  
print("nu.shape", nu.shape)  
print("outlier fraction", nu.shape[0]/target.shape[0])


In [ ]:
normal=ndf.loc[ndf.attack==1] #dataframe with normal observations 
abnormal=ndf.loc[ndf.attack==-1] #dataframe with abnormal observations



In [ ]:
# Once class SVM is trained with the observations of only one class.The algorithm is trained with 
# first 19000 observation of normal transactions. The remaining observation is merged with the anomalous observation 
# to create a test set. 
X_train = normal.loc[0:19000, :].drop('attack',1)

#creating test observations
X_test_1 = normal.loc[19001:, :].drop('attack',1)
X_test_2 = abnormal.drop('attack',1)
X_test = X_test_1.append(X_test_2)

#Y_test to test the output of model
Y_1 = normal.loc[19001:, 'attack']
Y_2 = abnormal['attack']
Y_test= Y_1.append(Y_2)

X_train_orig=X_train.copy()
X_test_orig=X_test.copy()



In [ ]:
#X_train=pd.get_dummies(X_train)
#X_test = pd.get_dummies(X_test)
#X_test=X_test.reindex(columns = X_train.columns, fill_value=0)


In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
# Iterating over all the common columns in train and test
for col in X_test.columns.values:
    # Encoding only categorical variables
    if X_test[col].dtypes=='object':
    # Using whole data to form an exhaustive list of levels
        data=X_train[col].append(X_test[col])
        le.fit(data.values)
        X_train[col]=le.transform(X_train[col])
        X_test[col]=le.transform(X_test[col])

In [ ]:
#Then encoding using one hot encoding
from sklearn.preprocessing import OneHotEncoder
columns=['access_method','user_name','user_agent_device','user_agent_os','user_agent_name','geoip_city_name','geoip_continent_name','geoip_country_iso_code','geoip_region_name','access_route']
enc=OneHotEncoder(sparse=False)


X_train_1=X_train
X_test_1=X_test

for col in columns:
    data=X_train[[col]].append(X_test[[col]])
    enc.fit(data.values)
    
    temp = enc.transform(X_train[[col]])
    # Changing the encoded features into a data frame with new column names
    temp=pd.DataFrame(temp,columns=[(col+"_"+str(i)) for i in data[col].value_counts().index])
    # Setting the index values similar to the X_train data frame
    temp=temp.set_index(X_train.index.values)
    # adding the new One Hot Encoded varibales to the train data frame
    X_train_1=pd.concat([X_train_1,temp],axis=1)
    
    # fitting One Hot Encoding on test data
    temp = enc.transform(X_test[[col]])
    # changing it into data frame and adding column names
    temp=pd.DataFrame(temp,columns=[(col+"_"+str(i)) for i in data[col].value_counts().index])
    # Setting the index for proper concatenation
    temp=temp.set_index(X_test.index.values)
    # adding the new One Hot Encoded varibales to test data frame
    X_test_1=pd.concat([X_test_1,temp],axis=1)

In [ ]:
#Drop the initial columns that are already encoded
X_train_1.drop(columns=columns,axis=1,inplace=True)
X_test_1.drop(columns=columns,axis=1,inplace=True)

In [ ]:
#standardadizing as SVM assumes standardization
from sklearn.preprocessing import scale
X_train_scale=scale(X_train_1)
X_test_scale=scale(X_test_1)

In [ ]:
# Setting the hyperparameters for Once Class SVM
oneclass = svm.OneClassSVM(kernel='rbf', gamma=0.004, nu=0.0001)

In [ ]:
#Fit the model with train_feature
oneclass.fit(X_train_scale)

In [ ]:
#Output of one class SVM
outlier_predict_test=oneclass.predict(X_test_scale)

In [ ]:
# Check the number of outliers predicted by the algorithm
unique, counts = np.unique(outlier_predict_test, return_counts=True)
print (np.asarray((unique, counts)).T)

In [ ]:
#Convert Y-test and outlier_predict to dataframe 

Y_test= Y_test.to_frame()
Y_test=Y_test.reset_index()
outlier_predict_test = pd.DataFrame(outlier_predict_test)
outlier_predict_test= outlier_predict_test.rename(columns={0: 'prediction'})

In [ ]:
#Calculate false positives and negatives
TP = FN = FP = TN = 0
for j in range(len(Y_test)):
    if Y_test['attack'][j]== 1 and outlier_predict_test['prediction'][j] == 1:
        TP = TP+1
    elif Y_test['attack'][j]== 1 and outlier_predict_test['prediction'][j] == -1:
        FN = FN+1
    elif Y_test['attack'][j]== -1 and outlier_predict_test['prediction'][j] == 1:
        FP = FP+1
    else:
        TN = TN +1
print (TP,  FN,  FP,  TN)

In [ ]:
from sklearn import metrics
print("accuracy:",metrics.accuracy_score(Y_test['attack'],outlier_predict_test['prediction']))
print("precison:",metrics.precision_score(Y_test['attack'],outlier_predict_test['prediction']))
print("recall:",metrics.recall_score(Y_test['attack'],outlier_predict_test['prediction']))
print("f1:",metrics.f1_score(Y_test['attack'],outlier_predict_test['prediction']))
print("area under curve (auc): ", metrics.roc_auc_score(Y_test['attack'],outlier_predict_test['prediction'])) 

In [ ]:
final_test_result_SVM=outlier_predict_test.set_index(X_test_orig.index.values).join(X_test_orig)

In [ ]:
#Predict the train results
outlier_predict_train=oneclass.predict(X_train_scale)

In [ ]:
#Convert train results to DataFrame
outlier_predict_train=pd.DataFrame(outlier_predict_train)
outlier_predict_train.rename(columns={0:'prediction'},inplace=True)

In [ ]:
X_train.hist(figsize=(20,20))
plt.show()

In [ ]:

corr=X_train.corr()

In [ ]:
import seaborn as sns
fig=plt.figure(figsize=(12,9))

In [ ]:
sns.heatmap(corr,vmax=0.8,square=True)
plt.show()

In [ ]:
from sklearn.ensemble import IsolationForest
IsoForest=IsolationForest(n_estimators=80,contamination=0.01,max_samples=len(X_train))

IsoForest.fit(X_train)
y_predict_train=IsoForest.predict(X_train)
score_train=IsoForest.decision_function(X_train)

unique, counts = np.unique(y_predict_train, return_counts=True)
print (np.asarray((unique, counts)).T)

In [ ]:
y_predict_train=pd.DataFrame(y_predict_train)
y_predict_train.rename(columns={0:'prediction'},inplace=True)

In [ ]:
y_predict_test=IsoForest.predict(X_test)
score_test=IsoForest.decision_function(X_test)

unique, counts = np.unique(y_predict_test, return_counts=True)
print (np.asarray((unique, counts)).T)

In [ ]:
#convert y_predict_test to DataFrame
y_predict_test=pd.DataFrame(y_predict_test)
y_predict_test.rename(columns={0:'prediction'},inplace=True)

In [ ]:
final_test_result_IF=y_predict_test.set_index(X_test.index.values).join(X_test_orig)

In [ ]:
if (X_test.index.values.all()==final_test_result_IF.index.values.all()):
    print("hello")

In [ ]:
X_train_orig['score_IF']=score_train

In [ ]:
X_train_orig['score_IF'][0]

In [ ]:
len(df[df['action']=='block'])

In [ ]:
a=final_test_result_SVM[final_test_result_SVM['prediction']==-1]

In [ ]:
b=final_test_result_IF[final_test_result_IF['prediction']==-1]